In [1]:
import math
import pandas as pd
from lenskit import topn
from sklearn.metrics import ndcg_score
from statistics import mean, variance
from lenskit.algorithms import Recommender
from lenskit.algorithms.user_knn import UserUser
from lenskit.metrics.topn import ndcg
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from lenskit.algorithms import Recommender
from lenskit.algorithms.user_knn import UserUser
from lenskit.metrics.topn import ndcg
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
preprocessed_dataset_folder = "../preprocessed_dataset"
ratings_df = pd.read_csv(preprocessed_dataset_folder+"/ratings.csv") 
movies_df = pd.read_csv(preprocessed_dataset_folder+"/movies.csv", index_col="item")

In [3]:
def pipeline(groupsize):
    
    users_ratings = ratings_df.groupby(['user']).count()
    selected = users_ratings['rating'] > 200
    selected_users = users_ratings.loc[selected]
    random_selected = selected_users.sample(groupsize) # sample() returns a random row from the dataframe. The returned object is a dataframe with one row. If you pass a number as argument you specify to select more than one row.
    select_column_df = random_selected.reset_index()['user'] # reset_index() create a new index, and the user became a column. Then, we can filter using the column name
    selected_users = select_column_df.values # iloc select by index, since our dataframe only has one row we read it from the index 0
    print("Selected users: " + str(selected_users))

    train_df, test_df = train_test_split(ratings_df, test_size=0.2)

    user_user = UserUser(15, min_nbrs=3)  # Minimum (3) and maximum (15) number of neighbors to consider
    recsys = Recommender.adapt(user_user)
    recsys.fit(train_df)

    test_df['predicted_rating'] = recsys.predict(test_df)

    test_df['relevant'] = test_df['rating'].apply(lambda x: 1 if x > 3 else 0)
    test_df['predicted_relevant'] = test_df['predicted_rating'].apply(lambda x: 1 if x > 3 else 0)

    train_df, test_df = train_test_split(ratings_df, test_size=0.2)

    user_user = UserUser(15, min_nbrs=3)  # Minimum (3) and maximum (15) number of neighbors to consider
    recsys = Recommender.adapt(user_user)
    recsys.fit(train_df)
    
    group_list = []
    for i in selected_users:
        selected_movies = recsys.recommend(i, 10)
        group_list.append(selected_movies)
        
    group_df = pd.concat(group_list)
    group_df.drop_duplicates(subset=["item"], inplace=True)
    
    lst = []
    for i in selected_users:
        grp = group_df.copy()
        grp["user"] = i
        grp["pred"] = recsys.predict_for_user(i, grp["item"]).values
        lst.append(grp)
        
    final = pd.concat(lst)
    #Here I implement both least misery and additive aggregations strategies
    final.sort_values(by=["pred"], inplace=True, ascending=False)

    gbadd = final.groupby("item", as_index=False)["pred"].sum() #Additive strat
    gbleast = final.groupby("item", as_index=False)["pred"].min() #Least misery strat

    final_recommendationsAdd = gbadd.sort_values(by=["pred"], ascending=False).head(5)
    final_recommendationsAdd
    
    final_recommendationsleast = gbleast.sort_values(by=["pred"], ascending=False).head(5)
    final_recommendationsleast
        
    evaluateAdd = final_recommendationsAdd.copy()
    for i in selected_users:
        evaluateAdd[i] = recsys.predict_for_user(i, final_recommendationsAdd["item"]).values
    evaluateAdd

    evaluateAdd.fillna(0, inplace=True)
    recommended_ordering = pd.DataFrame(evaluateAdd[["item","pred"]])
    recommended_ordering
        
    evaluateleast = final_recommendationsleast.copy()
    for i in selected_users:
        evaluateleast[i] = recsys.predict_for_user(i, final_recommendationsleast["item"]).values
    evaluateleast

    evaluateleast.fillna(0, inplace=True)
    recommended_orderingleast = pd.DataFrame(evaluateleast[["item","pred"]])
    recommended_orderingleast
    
    
    ndcg_lstAdd = []
    for i in selected_users:
        evaluateAdd.sort_values(by=[i], ascending=False, inplace=True)
        ev = pd.DataFrame(evaluateAdd[["item", i]])
        val = ndcg_score([ev.item], [recommended_ordering.item])
        ndcg_lstAdd.append(val)

    
    
    ndcg_lstleast = []
    for i in selected_users:
        evaluateleast.sort_values(by=[i], ascending=False, inplace=True)
        ev = pd.DataFrame(evaluateleast[["item", i]])
        val = ndcg_score([ev.item], [recommended_orderingleast.item])
        ndcg_lstleast.append(val)

    return mean(ndcg_lstAdd), mean(ndcg_lstleast)
    

        

In [4]:
gsize2_least = []
gsize2_add = []
for i in range(1000):
    gsize2_least.append(pipeline(4)[0])
    gsize2_add.append(pipeline(4)[1])
    
print(mean(gsize2_add))
print(mean(gsize2_least))

Selected users: [ 57 294 448 160]


BLAS using multiple threads - can cause oversubscription
found 1 potential runtime problems - see https://boi.st/lkpy-perf


Selected users: [560 514  51 226]
Selected users: [414 573 509  45]
Selected users: [382 603  28 305]
Selected users: [580 200  89 132]
Selected users: [381 567 288 292]
Selected users: [387 489 600 199]
Selected users: [477 381 182  91]
Selected users: [199 560 339  66]
Selected users: [477 599 156 391]
Selected users: [599 596 480  64]
Selected users: [ 42 514 298 249]
Selected users: [525 226 599 368]
Selected users: [382 274  57 603]
Selected users: [489 480 294  57]
Selected users: [469 480 156  45]
Selected users: [603 313  19 509]
Selected users: [610 414 381 509]
Selected users: [ 66  45 339 111]
Selected users: [600 514 313 339]
Selected users: [305 428 111 525]
Selected users: [561 567 307 305]
Selected users: [305 597 391 573]
Selected users: [484 477 318 600]
Selected users: [ 91 219 292 156]
Selected users: [177  91 597 275]
Selected users: [ 45 132  91 368]
Selected users: [534  21 368 428]
Selected users: [288 597  91 428]
Selected users: [202  68 307  42]
Selected users

Selected users: [ 42 534 600 318]
Selected users: [298 232 561 307]
Selected users: [ 18 610 525 391]
Selected users: [132 480 606  62]
Selected users: [ 91 200 132 469]
Selected users: [462 596   6 484]
Selected users: [382 391 462 103]
Selected users: [590 380 462  28]
Selected users: [160 307 103 156]
Selected users: [382 357 484 600]
Selected users: [298 199 484  91]
Selected users: [305 125 414 596]
Selected users: [140 428 517  18]
Selected users: [ 89  68 517 469]
Selected users: [597 428 514 232]
Selected users: [156 382 590 381]
Selected users: [474 357 288   6]
Selected users: [469 474 599 275]
Selected users: [232 438  66 368]
Selected users: [573 294 217 387]
Selected users: [474 382 288 307]
Selected users: [387 382 474 606]
Selected users: [603  68 509 561]
Selected users: [517 368 140 199]
Selected users: [  6 484 226 573]
Selected users: [288 517 217  28]
Selected users: [ 42 219 294 156]
Selected users: [202 140 438 199]
Selected users: [ 45 477 391 509]
Selected users

Selected users: [132 249 391 438]
Selected users: [217  66 517 274]
Selected users: [305  21 509  19]
Selected users: [489  51 555  19]
Selected users: [ 51 111 480 156]
Selected users: [275 517 606 469]
Selected users: [ 89 357 292 140]
Selected users: [ 45 514 182 509]
Selected users: [596 357 525 387]
Selected users: [232 489 474 382]
Selected users: [ 57 292 226 590]
Selected users: [448 567  91 561]
Selected users: [368 606 483  51]
Selected users: [596 603 567 608]
Selected users: [105 305 477 382]
Selected users: [219 477 294 232]
Selected users: [596 534 509 387]
Selected users: [599 357 414 603]
Selected users: [202 357 219 125]
Selected users: [382 391 226 596]
Selected users: [462 275 391  57]
Selected users: [474 484 489 313]
Selected users: [249 294 477  57]
Selected users: [318 599 610  66]
Selected users: [608 534 140 199]
Selected users: [597 608 249 217]
Selected users: [103 480  18 219]
Selected users: [590 382 226 567]
Selected users: [226 274 292 514]
Selected users

...

Selected users: [ 89  19 202 590]
Selected users: [ 45 428 597 232]
Selected users: [140 339 480 567]
Selected users: [182 202 599  28]
Selected users: [200  57 318 182]
Selected users: [561  19 298 474]
Selected users: [567  45 305 182]
Selected users: [606 132 474 275]
Selected users: [381 105  51 307]
Selected users: [200 368 599 580]
Selected users: [380 219 606 391]
Selected users: [534 140 525 484]
Selected users: [483 560  51 387]
Selected users: [391 318 292 125]
Selected users: [469 438 590 462]
Selected users: [200 318  42 428]
Selected users: [294 580 232   6]
Selected users: [274  68 182 368]
Selected users: [182 294 590 200]
Selected users: [561 313  51  19]
Selected users: [600 597  21 596]
Selected users: [600 483 560 489]
Selected users: [ 66 474 606 596]
Selected users: [ 19 509  28 606]
Selected users: [ 62 610 380 525]
Selected users: [484 534 448 368]
Selected users: [480 608 606 111]
Selected users: [590 105 599 428]
Selected users: [608 599 560 610]
Selected users

Selected users: [ 64  42  28 483]
Selected users: [381 599 514 275]
Selected users: [489 573 288 483]
Selected users: [202 232 274 199]
Selected users: [313  42 103 606]
Selected users: [160 125 307 103]
Selected users: [489 182 391 226]
Selected users: [380 590 561 275]
Selected users: [249 292 608  62]
Selected users: [381 597 219 606]
Selected users: [603 474  28 105]
Selected users: [ 68 182 469 483]
Selected users: [294 217 477 573]
Selected users: [387 339 603 292]
Selected users: [ 45 111 560 484]
Selected users: [514 318  45 448]
Selected users: [ 45  21 469  62]
Selected users: [608 606 275 105]
Selected users: [232  64 567 219]
Selected users: [200 474  68 597]
Selected users: [534 480 438 140]
Selected users: [318  28 177 357]
Selected users: [509 414  51 560]
Selected users: [603 509 608 217]
Selected users: [ 64 132  28  45]
Selected users: [597 381 580 387]
Selected users: [534  62  64  89]
Selected users: [368 573 339 292]
Selected users: [382 561 477 294]
Selected users

Selected users: [483 387 274 125]
Selected users: [28 89 62 19]
Selected users: [477 414  19 357]
Selected users: [132 596  91 111]
Selected users: [573 318 603  42]
Selected users: [525 288 307 219]
Selected users: [ 64 160 469 182]
Selected users: [125  45  51 448]
Selected users: [573 525 438  57]
Selected users: [305 382 525 307]
Selected users: [275  18 477 294]
Selected users: [357 480 606  18]
Selected users: [610 199  68  91]
Selected users: [275 380 105 382]
Selected users: [567 448 217   6]
Selected users: [561 483  57  18]
Selected users: [462  18 380 477]
Selected users: [382 307 573 469]
Selected users: [292 608 534  45]
Selected users: [560 414 567  89]
Selected users: [484 489 606 313]
Selected users: [217 600  66  28]
Selected users: [105 125 318 387]
Selected users: [140  28 103 514]
Selected users: [313  21 202 380]
Selected users: [292 560 603 380]
Selected users: [140  57 600 489]
Selected users: [525 182 580 438]
Selected users: [ 57 517 534  19]
Selected users: [4

Selected users: [105  21 517 480]
Selected users: [555  62 339 292]
Selected users: [200 202 182 489]
Selected users: [111 555  21 125]
Selected users: [103 597 249 514]
Selected users: [111 199 156 182]
Selected users: [462 380 368 567]
Selected users: [ 89 590 603 600]
Selected users: [ 66 606 232  57]
Selected users: [292 111 132 217]
Selected users: [477 600  66 428]
Selected users: [534 596 298 484]
Selected users: [448  68 298 200]
Selected users: [125 606 489 202]
Selected users: [200 199 555 274]
Selected users: [313  45 111 103]
Selected users: [160 561 489  28]
Selected users: [ 66  64 199 517]
Selected users: [132  21 298 483]
Selected users: [480 160 534 387]
Selected users: [232 177 381 480]
Selected users: [480 219 483 514]
Selected users: [ 51 381  57   6]
Selected users: [339 573 603 202]
Selected users: [428 103 298 160]
Selected users: [600  66  89 217]
Selected users: [ 28 590 217 274]
Selected users: [ 21 469 380 200]
Selected users: [381 474 606  68]
Selected users

Selected users: [156 202 489 274]
Selected users: [125 226 606 381]
Selected users: [474 462 414 217]
Selected users: [573 305 469  45]
Selected users: [483 313 140 202]
Selected users: [474 339  45 438]
Selected users: [103 305  21 573]
Selected users: [217 438 489 483]
Selected users: [105 182   6 489]
Selected users: [590 219 469 132]
Selected users: [140  18  68 560]
Selected users: [199 590 608  89]
Selected users: [567 111 160 534]
Selected users: [428 103 391 111]
Selected users: [469  18 555 156]
Selected users: [380 368 217 305]
Selected users: [177  89 232  91]
Selected users: [610 274 219 217]
Selected users: [217 288 555 200]
Selected users: [298  89 514 103]
Selected users: [  6 414 391 217]
Selected users: [ 89 599   6 534]
Selected users: [483  42  64 573]
Selected users: [  6 140 608 573]
Selected users: [480 573  91 567]
Selected users: [298 599 125 382]
Selected users: [313 177 597 525]
Selected users: [414 596 525 202]
Selected users: [391 448  42 438]
Selected users